# QUERIES

## SQL

In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Conexión a PostgreSQL
user = "userGESTDB"
password = "passGESTDB"
host = "postgres_db" 
port = "5432"
db = "GESTDB"

engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{db}")

A. Ver grados con su rama, área y universidad

In [2]:
df_1 = pd.read_sql(
"""
SELECT  
    g.id AS id_grado, 
    g.nombre AS grado, 
    a.nombre AS area, 
    r.nombre AS rama, 
    u.nombre AS universidad 

FROM grado g 
JOIN area a ON g.id_area = a.id 
JOIN rama r ON a.id_rama = r.id 
JOIN grado_ofertado go ON g.id = go.id_grado 
JOIN facultad f ON go.id_facultad = f.id 
JOIN universidad u ON f.id_universidad = u.id 
ORDER BY r.nombre, g.nombre;
""", engine)
df_1.head()

,id_grado,grado,area,rama,universidad
0,52,Antropología Social y Cultural,Antropología Social y Cultural,Artes y Humanidades,Universidad Europea de Madrid
1,1,Antropología Social y Cultural,Antropología Social y Cultural,Artes y Humanidades,Universidad Autónoma de Madrid
2,1,Antropología Social y Cultural,Antropología Social y Cultural,Artes y Humanidades,Universidad Nacional de Educación a Distancia ...
3,1,Antropología Social y Cultural,Antropología Social y Cultural,Artes y Humanidades,Universidad Complutense de Madrid
4,32,Arqueología,Geografía e Historia,Artes y Humanidades,Universidad Nacional de Educación a Distancia ...


B. Consulta de titulaciones posibles según la nota de admisión

In [3]:
df_2 = pd.read_sql(
    """
SELECT  
    g.nombre AS grado, 
    u.nombre AS universidad, 
    nc.nota AS nota_corte, 
    nc.año, 
    nc.convocatoria 
FROM nota_corte nc 
JOIN grado_ofertado go ON nc.id_grado_ofertado = go.id 
JOIN grado g ON go.id_grado = g.id 
JOIN facultad f ON go.id_facultad = f.id 
JOIN universidad u ON f.id_universidad = u.id 
WHERE nc.nota <= 11.0 

ORDER BY nc.nota DESC; 
    """, engine)
df_2.head()

,grado,universidad,nota_corte,año,convocatoria
0,Ingeniería en Tecnologías Industriales,Universidad Carlos III de Madrid,11.0,2025,ordinaria
1,Ingeniería en Tecnologías Industriales,Universidad a Distancia de Madrid UDIMA,11.0,2025,ordinaria
2,Ingeniería en Tecnologías Industriales,Universidad de Alcalá,11.0,2025,ordinaria
3,Ingeniería en Tecnologías Industriales,Universidad Politécnica de Madrid,11.0,2025,ordinaria
4,Ingeniería en Tecnologías Industriales,Universidad Pontificia Comillas ICAIICADE,11.0,2025,ordinaria


C. Consulta de universidades públicas por rama de conocimiento

In [4]:
df_3 = pd.read_sql(
    """
SELECT DISTINCT  
    u.nombre AS universidad, 
    r.nombre AS rama 

FROM universidad u 
JOIN facultad f ON u.id = f.id_universidad 
JOIN grado_ofertado go ON f.id = go.id_facultad 
JOIN grado g ON go.id_grado = g.id 
JOIN area a ON g.id_area = a.id 
JOIN rama r ON a.id_rama = r.id 

WHERE u.tipo = 'Pública' 
  AND r.nombre = 'Ingeniería y Arquitectura' 
ORDER BY u.nombre; 
    """, engine)
df_3.head()

,universidad,rama
0,Universidad Autónoma de Madrid,Ingeniería y Arquitectura
1,Universidad Carlos III de Madrid,Ingeniería y Arquitectura
2,Universidad Complutense de Madrid,Ingeniería y Arquitectura
3,Universidad de Alcalá,Ingeniería y Arquitectura
4,Universidad Nacional de Educación a Distancia ...,Ingeniería y Arquitectura


D. Comparativa de grados similares según nota media

In [13]:
df_4 = pd.read_sql(
    """
SELECT  
    a.nombre AS area, 
    g.nombre AS grado, 
    ROUND(AVG(nc.nota), 2) AS nota_media 

FROM grado g 
JOIN area a ON g.id_area = a.id 
JOIN grado_ofertado go ON g.id = go.id_grado 
JOIN nota_corte nc ON go.id = nc.id_grado_ofertado 
GROUP BY a.nombre, g.nombre 

ORDER BY a.nombre, nota_media DESC; 
    """, engine)
df_4.head()

,area,grado,nota_media
0,Antropología Social y Cultural,Antropología Social y Cultural,5.00
1,Arquitectura,Fundamentos de Arquitectura y Urbanismo,8.64
2,Artes,Conservación y Restauración del Patrimonio Cul...,6.71
3,Artes,Bellas Artes,6.32
4,Artes,Composición Musical,5.00


E. Consultar grados de una misma facultad

In [27]:
df_5 = pd.read_sql(
    """
    SELECT  
        g.id AS id_grado, 
        g.nombre AS grado,
        u.nombre AS universidad,
        f.nombre AS factultad
    FROM universidad u
    JOIN facultad f ON f.id_universidad = u.id 
    JOIN grado_ofertado go ON go.id_facultad = f.id
    JOIN grado g ON g.id = go.id_grado
    WHERE f.id= 9
    ORDER BY g.nombre;
    """,
    engine
)

df_5.head()


,id_grado,grado,universidad,factultad
0,108,Bachelor of Urban Management,Universidad San PabloCEU,Facultad de Humanidades y Ciencias de la Comun...
1,109,Estudios Urbanos,Universidad San PabloCEU,Facultad de Humanidades y Ciencias de la Comun...
2,33,Geografía e Historia,Universidad San PabloCEU,Facultad de Humanidades y Ciencias de la Comun...
3,34,Historia,Universidad San PabloCEU,Facultad de Humanidades y Ciencias de la Comun...
4,111,Información y Documentación,Universidad San PabloCEU,Facultad de Humanidades y Ciencias de la Comun...


F. Consultar facultades de una universidad

In [28]:
df_6 = pd.read_sql(
    """
    SELECT  
        u.nombre AS universidad,
        f.nombre AS factultad
    FROM universidad u
    JOIN facultad f ON f.id_universidad = u.id 
    WHERE u.id= 1
    ORDER BY f.nombre;
    """,
    engine
)

df_6.head()


,universidad,factultad
0,Universidad Nacional de Educación a Distancia ...,Escuela Técnica Superior de Ingenieros Industr...
1,Universidad Nacional de Educación a Distancia ...,Facultad Ciencias Económicas y Empresariales
2,Universidad Nacional de Educación a Distancia ...,Facultad de Ciencias
3,Universidad Nacional de Educación a Distancia ...,Facultad de Ciencias Económicas y Empresariales
4,Universidad Nacional de Educación a Distancia ...,Facultad de Ciencias Políticas y Sociología


## Elasticsearch

In [29]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://elasticsearch:9200")

# Verifica la conexión
print(es.info().body)

{'name': 'f9d659a396d8', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'QjqgR16TSLem8J1oCtOMtQ', 'version': {'number': '8.7.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '09520b59b6bc1057340b55750186466ea715e30e', 'build_date': '2023-03-27T16:31:09.816451435Z', 'build_snapshot': False, 'lucene_version': '9.5.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [30]:
# creamos los embedings de la descripcion de titulo
from sentence_transformers import SentenceTransformer

# Load a pre-trained Sentence Transformer model
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Generate embeddings (vector representations) for descripciones
embeddings_descripcion = model.encode(df["descripcion"].values.tolist())

# Generate embeddings (vector representations) for salidas
embeddings_salidas = model.encode(df["salidas"].values.tolist())

/opt/conda/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


NameError: name 'df' is not defined

In [ ]:
import json
from elasticsearch.helpers import bulk

index_name="informacion_grados"

def vector_bulk_index_data(es, data, index_name):
    batch_size = 50  # Reducir el tamaño del lote a 50
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        actions = []
        for idx, doc in enumerate(batch):
            doc["descripcion_vector"]=embeddings_descripcion[i + idx].tolist()
            doc["salidas_vector"]=embeddings_salidas[i + idx].tolist()
            actions.append({
                "_index": index_name,
                "_id": doc['id'],
                "_source": doc
            })
        # Capturar la respuesta para verificar errores
        resp = bulk(es, actions, raise_on_error=True)
        print("Indexed:", resp[0], "Errors:", resp[1])

descripciones = df.to_dict(orient='records')
vector_bulk_index_data(es, descripciones, index_name)

### Queries sintácticas

In [ ]:
import json

res = es.search(
    index=index_name,
    size=3,
    query={
        "term": {
            "nombre": "industrial"
        }
    }
)

print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

In [ ]:
res = es.search(
    index=index_name,
    size=3,
    query={
        "term": {
            "nombre": "informática"
        }
    }
)

print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

### Queries léxcias

In [ ]:
res = es.search(
    index="informacion_grados",
    size=3,
    query={
        "match": {
            "descripcion": "computación"
        }
    }
)


print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

In [ ]:
res = es.search(
    index="informacion_grados",
    size=3,
    query={
        "bool": {
            "must": [
                {"match": {"descripcion": "comunicación"}},
                {"match": {"salidas": "medios audiovisuales"}}
            ],
            "must_not": [
                {"match": {"descripcion": "ingeniería"}},
                {"match": {"descripcion": "matemáticas"}}
            ]
        }
    }
)

print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

In [ ]:
res = es.search(
    index="informacion_grados",
    size=1,
    query={
        "bool": {
            "must": [
                {"match": {"descripcion": "gestión"}},
                {"match": {"salidas": "administración de empresas"}}
            ],
            "must_not": [
                {"match": {"descripcion": "finanzas"}},
                {"match": {"salidas": "contabilidad"}}
            ]
        }
    }
)

print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

In [ ]:
res = es.search(
    index="informacion_grados",
    size=3,
    query={
        "bool": {
            "must": [
                {"match": {"descripcion": "comportamiento humano"}},
                {"match": {"salidas": "social"}}
            ],
            "must_not": [
                {"term": {"nombre.keyword": "Psicología"}},
                {"term": {"nombre.keyword": "Biología Sanitaria"}}
            ]
        }
    }
)

print("Search Results:")
for hit in res['hits']['hits']:
    print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
    print("")

In [ ]:
res = es.search(
    index="informacion_grados",
    size=2,
    query={
        "bool": {
            "must": [
                {"match": {"descripcion": "arte digital"}},
                {"match": {"salidas": "diseño"}}
            ],
            "must_not": [
                {"match": {"descripcion": "docencia"}},
                {"match": {"descripcion": "ingeniería"}}
            ]
        }
    }
)

print("Search Results:")
for hit in res['hits']['hits']:
    print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
    print("")


### Queries semánticas

In [ ]:
query_sentence = "análisis de datos"
query_vector = model.encode([query_sentence])[0]

parameters = {
     "field":"descripcion_vector",
     "query_vector": query_vector,
     "k":5,
     "num_candidates":100
}
res = es.search(
    index=index_name, 
    knn=parameters)

print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

## Graph/Sparql